In [1]:
import pandas as pd
from glob import glob


SUB_DIR  = "/playpen-ssd/levi/w4c/w4c-25/__exps__/_BEST/submission-bins-1-all-regions"
fps = glob(f"{SUB_DIR}/*/*.csv")

In [2]:
import os
import subprocess

from pathlib import Path
from src.dataloader.challenge_one_dataloader import Sat2RadDataset
from torch.utils.data import DataLoader


outdir     = "/playpen-ssd/levi/w4c/w4c-25/__exps__/_BEST/variations"
dataset    = Sat2RadDataset(split="test")
dataloader = DataLoader(dataset, num_workers=0, batch_size=1)


# 0, 16, 32
for num_buckets in [4, 8, 16, 32, 64]:

    submission_dir = Path(outdir) / Path(f"num_buckets_{num_buckets}")
    os.makedirs(submission_dir, exist_ok=True)

    sub_dir_2019   = Path(submission_dir) / Path("2019")
    sub_dir_2020   = Path(submission_dir) / Path("2020")

    os.makedirs(sub_dir_2019, exist_ok=True)
    os.makedirs(sub_dir_2020, exist_ok=True)

    preds = {}

    for fp in fps:
        
        df    = pd.read_csv(fp)
        
        # original predictions/emperical distribution
        og_preds = {}

        for row in df.iterrows():
            
            case, bucket, cum_val = row[1][0], row[1][1], row[1][2]
            if case not in og_preds:
                og_preds[case] = {
                    "buckets" : [],
                    "cum_vals": [],
                }
            og_preds[case]["buckets"].append(bucket)
            og_preds[case]["cum_vals"].append(cum_val)

        sparse_cases    = []
        sparse_buckets  = []
        sparse_cum_vals = []

        for k, v in og_preds.items():
 
            _buckets  = og_preds[k]['buckets'][::(128  // num_buckets)]
            _cum_vals = og_preds[k]['cum_vals'][::(128 // num_buckets)]
            _cases    = [k] * len(_cum_vals)

            sparse_cases.extend(_cases)
            sparse_buckets.extend(_buckets)
            sparse_cum_vals.extend(_cum_vals)

        out_fp = Path(submission_dir) / Path(fp).parent.name / Path(fp).name
        preds[str(out_fp)] = [[case, bucket, val] for case, bucket, val in zip(sparse_cases, sparse_buckets, sparse_cum_vals)]

    # save preds to csv
    for k, v in preds.items():
        df = pd.DataFrame(v)
        df.to_csv(k, index=False, header=False)

    # cd into exp dir
    os.chdir(str(submission_dir))

    # store as zip
    command = ["zip", "-r", f"../num-buckets-{num_buckets}-all-regions.zip", ".", "i", "*", "&", "exit"]
    subprocess.run(command)

Loading dataset...: 100%|██████████| 3/3 [00:00<00:00, 140.29it/s]


	zip warning: name not matched: i
	zip warning: name not matched: *
	zip warning: name not matched: &
	zip warning: name not matched: exit
  adding: 2019/ (stored 0%)
  adding: 2019/roxi_0010.test.cum4h.csv (deflated 60%)
  adding: 2019/roxi_0009.test.cum4h.csv (deflated 63%)
  adding: 2019/roxi_0008.test.cum4h.csv (deflated 72%)
  adding: 2020/ (stored 0%)
  adding: 2020/roxi_0010.test.cum4h.csv (deflated 59%)
  adding: 2020/roxi_0009.test.cum4h.csv (deflated 56%)
  adding: 2020/roxi_0008.test.cum4h.csv (deflated 58%)
	zip warning: name not matched: i
	zip warning: name not matched: *
	zip warning: name not matched: &
	zip warning: name not matched: exit
  adding: 2019/ (stored 0%)
  adding: 2019/roxi_0010.test.cum4h.csv (deflated 60%)
  adding: 2019/roxi_0009.test.cum4h.csv (deflated 63%)
  adding: 2019/roxi_0008.test.cum4h.csv (deflated 72%)
  adding: 2020/ (stored 0%)
  adding: 2020/roxi_0010.test.cum4h.csv (deflated 59%)
  adding: 2020/roxi_0009.test.cum4h.csv (deflated 58%)
  add

In [3]:
fp, og_preds.keys()

('/playpen-ssd/levi/w4c/w4c-25/__exps__/_BEST/submission-bins-1-all-regions/2020/roxi_0008.test.cum4h.csv',
 dict_keys(['RIG3', 'TAL3', 'TUR4', 'WAW2', 'RIG4', 'TAL4', 'SPT4', 'WAW3', 'JYV4', 'OUL5', 'OUL6', 'WAW4', 'HEL4', 'OUL7']))

In [1]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt


x = torch.load("/playpen-ssd/levi/w4c/w4c-25/___old___/11-4-25-y_reg_norms.pth")
x_arr = x.numpy()
random.shuffle(x_arr)

In [2]:
ss = x_arr[:32000]
ss_rescaled = ss * 13.8070

In [7]:
import matplotlib.ticker as ticker


fig, axes = plt.subplots(1, 1, figsize=(12, 5), constrained_layout=False, dpi=1200)

color_palette = {
    # "unc_primary": "#4B9CD3",
    "unc_primary": "#EDE9FF",
    "unc_secondary": "#13294B",
}
axes.set_facecolor("white")
axes.grid(True, axis="y", alpha=0.25, linestyle='--')
axes.set_xlabel("Mean cumulative hourly precipitation (mm/hr)", fontsize=14)
axes.set_ylabel("Count", fontsize=14)
axes.hist(
    ss_rescaled, 
    bins=64, 
    log=True, 
    # histtype="bar", 
    color=color_palette['unc_primary'], 
    # edgecolor=color_palette['unc_secondary'], 
    edgecolor="black",
    linewidth=0.5
);

axes.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
axes.tick_params(axis='both', which='both', labelsize=12, direction='in')